# MARec++: Full Enhancement Suite
## Cold-Start Recommendation with 6 Enhancement Modules

**Enhancements:** CA-Rec, UA-Rec, GE-Rec, Diff-Rec, HCA, UADA, Learned Fusion

| Mode | Config | Time |
|------|--------|------|
| **TURBO** | 1×1×6 | ~8 min |
| **FAST** | 3×2×6 | ~25 min |

---
## 1. Configuration

In [ ]:
import time as _time
_NOTEBOOK_START = _time.time()

TURBO_MODE = True  # True=~8 min, False=~25 min

if TURBO_MODE:
    CONFIG = {
        'seed': 42, 'n_splits': 1, 'n_seeds': 1,
        'dataset': 'hetrec', 'cold_train_frac': 0.60, 'cold_val_frac': 0.20,
        'lambda1': 1.0, 'alpha': 1.0, 'beta': 100.0, 'delta': 20.0, 'second_order': True,
        'use_st': True, 'st_model': 'all-MiniLM-L6-v2',
        'use_ca_rec': True, 'ca_epochs': 5, 'ca_lr': 2e-3, 'ca_hidden_dim': 64, 'ca_temperature': 0.07,
        'use_hca': True, 'hca_hard_neg_k': 10,
        'use_ua_rec': True, 'ua_epochs': 5, 'ua_lr': 2e-3, 'ua_hidden_dim': 64,
        'use_uada': True, 'uada_epochs': 5, 'uada_lr': 2e-3,
        'use_ge_rec': True, 'ge_latent_dim': 16, 'ge_hidden_dim': 64, 'ge_epochs': 8, 'ge_lr': 3e-3, 'ge_kl_warmup': 3, 'ge_kl_weight': 0.01,
        'use_diff_rec': True, 'diff_steps': 5, 'diff_epochs': 5, 'diff_lr': 2e-3, 'diff_hidden_dim': 64,
        'use_learned_fusion': True, 'fusion_hidden_dim': 32,
        'ks': [10, 50], 'output_dir': '/content/marec_results',
    }
    print('⚡ TURBO MODE: ~8 min')
else:
    CONFIG = {
        'seed': 42, 'n_splits': 3, 'n_seeds': 2,
        'dataset': 'hetrec', 'cold_train_frac': 0.60, 'cold_val_frac': 0.20,
        'lambda1': 1.0, 'alpha': 1.0, 'beta': 100.0, 'delta': 20.0, 'second_order': True,
        'use_st': True, 'st_model': 'all-MiniLM-L6-v2',
        'use_ca_rec': True, 'ca_epochs': 10, 'ca_lr': 1e-3, 'ca_hidden_dim': 128, 'ca_temperature': 0.07,
        'use_hca': True, 'hca_hard_neg_k': 20,
        'use_ua_rec': True, 'ua_epochs': 10, 'ua_lr': 1e-3, 'ua_hidden_dim': 128,
        'use_uada': True, 'uada_epochs': 10, 'uada_lr': 1e-3,
        'use_ge_rec': True, 'ge_latent_dim': 32, 'ge_hidden_dim': 128, 'ge_epochs': 15, 'ge_lr': 2e-3, 'ge_kl_warmup': 5, 'ge_kl_weight': 0.01,
        'use_diff_rec': True, 'diff_steps': 10, 'diff_epochs': 10, 'diff_lr': 1e-3, 'diff_hidden_dim': 128,
        'use_learned_fusion': True, 'fusion_hidden_dim': 64,
        'ks': [10, 25, 50], 'output_dir': '/content/marec_results',
    }
    print(f'🚀 FAST MODE: ~25 min')

---
## 2. Environment

In [ ]:
import subprocess, sys, os, time, warnings, gc, random, math
from collections import defaultdict
from itertools import product as iprod

def install(pkg): subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
for pkg in ['scipy', 'scikit-learn', 'pandas', 'matplotlib', 'seaborn', 'tqdm']: install(pkg)
try: import sentence_transformers
except: install('sentence-transformers')

import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, normalize
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.nn.functional as F_t
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

def set_seed(s): random.seed(s); np.random.seed(s); torch.manual_seed(s)
set_seed(CONFIG['seed'])
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

---
## 3. Data

In [ ]:
import urllib.request, zipfile
DATA_DIR = '/content/data/hetrec'
CACHE_DIR = '/content/cache'
os.makedirs(DATA_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)
URL = 'https://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip'

def find_file(base, name):
    for r, _, f in os.walk(base):
        if name in f: return os.path.join(r, name)
    return None

if find_file(DATA_DIR, 'user_ratedmovies.dat') is None:
    print('📥 Downloading...'); zp = os.path.join(DATA_DIR, 'h.zip')
    urllib.request.urlretrieve(URL, zp)
    with zipfile.ZipFile(zp, 'r') as z: z.extractall(DATA_DIR)
    os.remove(zp)
print('✓ Data ready')

In [ ]:
rf = find_file(DATA_DIR, 'user_ratedmovies.dat') or find_file(DATA_DIR, 'user_ratedmovies-timestamps.dat')
raw = pd.read_csv(rf, sep='\t', encoding='latin-1')
ratings = raw[['userID', 'movieID']].drop_duplicates()

all_users = sorted(ratings['userID'].unique()); all_items = sorted(ratings['movieID'].unique())
user2idx = {u: i for i, u in enumerate(all_users)}; item2idx = {it: i for i, it in enumerate(all_items)}
idx2item = {i: it for it, i in item2idx.items()}
n_users, n_items = len(all_users), len(all_items)

row = ratings['userID'].map(user2idx).values; col = ratings['movieID'].map(item2idx).values
X = csr_matrix((np.ones(len(ratings)), (row, col)), shape=(n_users, n_items))

metadata = {}
for fn, k, cn in [('movie_genres.dat', 'genres', 'genre'), ('movie_countries.dat', 'countries', 'country')]:
    f = find_file(DATA_DIR, fn)
    if f: df = pd.read_csv(f, sep='\t', encoding='latin-1'); metadata[k] = df.groupby('movieID')[cn].apply(list).to_dict()
f = find_file(DATA_DIR, 'movie_directors.dat')
if f: df = pd.read_csv(f, sep='\t', encoding='latin-1'); metadata['directors'] = df.groupby('movieID').apply(lambda x: x.iloc[:, 1].tolist()).to_dict()
f = find_file(DATA_DIR, 'movies.dat')
if f:
    try: df = pd.read_csv(f, sep='\t', encoding='latin-1'); metadata['titles'] = df.set_index(df.columns[0])['title'].to_dict() if 'title' in df.columns else {}
    except: pass

print(f'✓ {n_users} users, {n_items} items, {X.nnz} interactions')

---
## 4. Features

In [ ]:
def build_fmats(metadata, item2idx, n_items):
    fmats = {}
    for k in ['genres', 'directors', 'countries']:
        if k not in metadata: continue
        mlb = MultiLabelBinarizer(sparse_output=True)
        labels = [[str(v) for v in metadata.get(k, {}).get(idx2item.get(i, i), [])] for i in range(n_items)]
        fmats[k] = csr_matrix(mlb.fit_transform(labels)); print(f'  {k}: {fmats[k].shape[1]}')
    return fmats

print('Building features...')
fmats = build_fmats(metadata, item2idx, n_items)

In [ ]:
ST_CACHE = os.path.join(CACHE_DIR, 'st_emb.npy')
if CONFIG['use_st']:
    if os.path.exists(ST_CACHE): st_emb = np.load(ST_CACHE); print('📂 Loaded cached ST')
    else:
        print(f'🧠 Encoding with {CONFIG["st_model"]}...')
        from sentence_transformers import SentenceTransformer
        _st = SentenceTransformer(CONFIG['st_model'])
        texts = ['. '.join([str(metadata.get('titles', {}).get(idx2item.get(i, i), ''))] + [', '.join(str(v) for v in metadata.get(k, {}).get(idx2item.get(i, i), [])[:3]) for k in ['genres', 'directors']]) for i in range(n_items)]
        st_emb = _st.encode(texts, show_progress_bar=True, batch_size=128)
        np.save(ST_CACHE, st_emb); del _st; gc.collect()
    fmats['st_emb'] = csr_matrix(st_emb); print(f'✓ ST: {st_emb.shape}')
print(f'Total: {sum(v.shape[1] for v in fmats.values())} dims')

In [ ]:
def create_splits(X, n, seed, tf=0.6, vf=0.2):
    rng = np.random.RandomState(seed); ni = X.shape[1]; splits = []
    for _ in range(n):
        perm = rng.permutation(ni); nt, nv = int(ni*tf), int(ni*vf)
        tr, va, te = sorted(perm[:nt].tolist()), sorted(perm[nt:nt+nv].tolist()), sorted(perm[nt+nv:].tolist())
        splits.append({'X_train': X[:, tr], 'X_test': X[:, te], 'train_items': tr, 'test_items': te, 'test_users': np.array(X[:, te].sum(1)).flatten() > 0})
    return splits

splits = create_splits(X, CONFIG['n_splits'], CONFIG['seed'])
print(f'✓ {len(splits)} splits, Train: {len(splits[0]["train_items"])}, Test: {len(splits[0]["test_items"])}')

---
## 5. Models

In [ ]:
class EASE:
    def __init__(self, l1=1.0): self.l1 = l1; self.B = None
    def fit(self, X, align=None):
        n = X.shape[1]; G = (X.T @ X).toarray().astype(np.float64)
        XtA = np.zeros_like(G) if align is None else (X.T.toarray().astype(np.float64) @ align)
        P = np.linalg.inv(G + self.l1*np.eye(n) + XtA); T = P @ (G + XtA)
        dP = np.diag(P).copy(); dP[np.abs(dP)<1e-10] = 1e-10
        self.B = T - P * (np.diag(T)/dP)[None, :]; return self
    def predict(self, X): return (X.toarray() if sp.issparse(X) else X) @ self.B

class MARecAligner:
    def __init__(self, a=1.0, b=100.0, d=20.0): self.a, self.b, self.d = a, b, d; self.mu = None; self.names = []
    def _d(self, M): return M.toarray() if sp.issparse(M) else np.asarray(M)
    def compute_G(self, fmats):
        G_list = []; self.names = list(fmats.keys())
        for n in self.names: Fd = self._d(fmats[n]); Fn = Fd / np.maximum(np.linalg.norm(Fd, axis=1, keepdims=True), 1e-10); G_list.append(Fn @ Fn.T)
        return G_list
    def fit_weights(self, Xtr, G_list): self.mu = np.ones(len(G_list))
    def combine_G(self, G_list): return sum(self.mu[k]*G_list[k] for k in range(len(G_list)))
    def cross_sim(self, fmats, cold, warm): return sum(self.mu[k]*(normalize(self._d(fmats[n][cold])) @ normalize(self._d(fmats[n][warm])).T) for k, n in enumerate(self.names))
    def compute_DR(self, Xtr): c = np.array(Xtr.sum(0)).flatten(); p = max(np.percentile(c[c>0], 10), 1); return np.diag(np.where(c<=p, (self.b/p)*np.maximum(p-c, 0), 0.0))
print('✓ EASE + MARec')

In [ ]:
class CARec(nn.Module):
    def __init__(self, md, id, hd=64): super().__init__(); self.mp = nn.Sequential(nn.Linear(md, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd)); self.ip = nn.Sequential(nn.Linear(id, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd))
    def forward(self, mf, vf, temp=0.07, hard_neg_k=0):
        m = F_t.normalize(self.mp(mf), dim=-1); v = F_t.normalize(self.ip(vf), dim=-1)
        logits = m @ v.T / temp
        if hard_neg_k > 0:  # HCA: hard negative mining
            with torch.no_grad(): neg_scores = logits.clone(); neg_scores.fill_diagonal_(-1e9); _, hard_idx = neg_scores.topk(hard_neg_k, dim=1)
            mask = torch.zeros_like(logits); mask.scatter_(1, hard_idx, 1.0); mask.fill_diagonal_(1.0)
            logits = logits * mask + (1 - mask) * (-1e9)
        return F_t.cross_entropy(logits, torch.arange(m.shape[0], device=m.device)), m
    @torch.no_grad()
    def project(self, mf): return F_t.normalize(self.mp(mf), dim=-1)
print('✓ CA-Rec + HCA')

In [ ]:
class UARec(nn.Module):
    def __init__(self, md, td, hd=64): super().__init__(); self.sh = nn.Sequential(nn.Linear(md, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd), nn.LayerNorm(hd), nn.GELU()); self.mu = nn.Linear(hd, td); self.lv = nn.Linear(hd, td)
    def forward(self, mf, tgt): h = self.sh(mf); mu = self.mu(h); lv = self.lv(h).clamp(-10, 2); return ((1.0/lv.exp())*(tgt-mu).pow(2) + lv).mean(), mu, lv.exp()
    @torch.no_grad()
    def predict(self, mf): h = self.sh(mf); return self.mu(h), self.lv(h).clamp(-10, 2).exp()

class UADA(nn.Module):  # Domain adaptation warm->cold
    def __init__(self, md, hd=64): super().__init__()
        self.enc = nn.Sequential(nn.Linear(md, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd))
        self.disc = nn.Sequential(nn.Linear(hd, hd//2), nn.GELU(), nn.Linear(hd//2, 1))
    def forward(self, warm, cold):
        hw = self.enc(warm); hc = self.enc(cold)
        lw = F_t.binary_cross_entropy_with_logits(self.disc(hw), torch.ones(hw.shape[0], 1, device=hw.device))
        lc = F_t.binary_cross_entropy_with_logits(self.disc(hc), torch.zeros(hc.shape[0], 1, device=hc.device))
        return lw + lc, hw, hc
print('✓ UA-Rec + UADA')

In [ ]:
class GERec(nn.Module):
    def __init__(self, id, md, ld=16, hd=64): super().__init__(); self.ld = ld
        self.enc = nn.Sequential(nn.Linear(id+md, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd), nn.LayerNorm(hd), nn.GELU())
        self.muz = nn.Linear(hd, ld); self.lvz = nn.Linear(hd, ld)
        self.dec = nn.Sequential(nn.Linear(ld+md, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, id))
    def forward(self, v, m, kl_w=0.01):
        h = self.enc(torch.cat([v, m], -1)); mz = self.muz(h); lz = self.lvz(h)
        z = mz + torch.exp(0.5*lz)*torch.randn_like(lz) if self.training else mz
        vh = self.dec(torch.cat([z, m], -1)); rec = F_t.mse_loss(vh, v); kl = -0.5*torch.mean(1+lz-mz.pow(2)-lz.exp())
        return rec + kl_w*kl, rec, kl, vh
    @torch.no_grad()
    def generate(self, m): return self.dec(torch.cat([torch.randn(m.shape[0], self.ld, device=m.device), m], -1))
print('✓ GE-Rec')

In [ ]:
class DiffRec(nn.Module):
    def __init__(self, ed, md, ns=5, hd=64): super().__init__(); self.ns = ns; self.ed = ed
        s = 0.008; t = torch.linspace(0, ns, ns+1); ab = torch.cos(((t/ns)+s)/(1+s)*math.pi*0.5)**2; ab = ab/ab[0]
        self.register_buffer('ab', ab[1:]); self.register_buffer('a', ab[1:]/ab[:-1]); self.register_buffer('b', 1-self.a)
        self.dn = nn.Sequential(nn.Linear(ed+md+1, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, hd), nn.LayerNorm(hd), nn.GELU(), nn.Linear(hd, ed))
    def forward(self, x0, m):
        t = torch.randint(0, self.ns, (x0.shape[0],), device=x0.device); n = torch.randn_like(x0)
        abt = self.ab[t].unsqueeze(1); xt = torch.sqrt(abt)*x0 + torch.sqrt(1-abt)*n
        return F_t.mse_loss(self.dn(torch.cat([xt, m, t.float().unsqueeze(1)/self.ns], -1)), n)
    @torch.no_grad()
    def denoise(self, xT, m, traj=False):
        x = xT; tr = [x.cpu().numpy()] if traj else None
        for t in reversed(range(self.ns)):
            tn = torch.full((x.shape[0],1), t/self.ns, device=x.device)
            pn = self.dn(torch.cat([x, m, tn], -1)); x = (1/torch.sqrt(self.a[t]))*(x-(self.b[t]/torch.sqrt(1-self.ab[t]))*pn)
            if t > 0: x = x + torch.sqrt(self.b[t])*torch.randn_like(x)
            if traj: tr.append(x.cpu().numpy())
        return (x, tr) if traj else x
print('✓ Diff-Rec')

In [ ]:
class LearnedFusion(nn.Module):
    def __init__(self, n_sources=4, hd=32): super().__init__()
        self.gate = nn.Sequential(nn.Linear(n_sources, hd), nn.GELU(), nn.Linear(hd, n_sources), nn.Softmax(dim=-1))
    def forward(self, scores_list):
        st = torch.stack(scores_list, dim=-1)  # (U, I, n_sources)
        g = self.gate(st)  # per-item gates
        return (st * g).sum(dim=-1)
print('✓ Learned Fusion')

In [ ]:
def evaluate(sc, Xte, ks=(10, 50), um=None):
    Xt = Xte.toarray() if sp.issparse(Xte) else Xte
    res = {f'hr@{k}': 0.0 for k in ks}; res.update({f'ndcg@{k}': 0.0 for k in ks}); ne = 0
    for u in range(Xt.shape[0]):
        if um is not None and not um[u]: continue
        t = np.where(Xt[u]>0)[0]
        if len(t)==0: continue
        rk = np.argsort(sc[u])[::-1]; ts = set(t)
        for k in ks:
            tk = rk[:k]; h = sum(1 for i in tk if i in ts); res[f'hr@{k}'] += h/min(k, len(t))
            dcg = sum(1.0/np.log2(r+2) for r, i in enumerate(tk) if i in ts); idcg = sum(1.0/np.log2(i+2) for i in range(min(k, len(t))))
            res[f'ndcg@{k}'] += (dcg/idcg) if idcg>0 else 0.0
        ne += 1
    for k in res: res[k] /= max(ne, 1)
    return res

def coverage(sc, k=50): return len(set(np.argsort(sc, axis=1)[:, -k:].flatten()))/sc.shape[1]
print('✓ Evaluation')

---
## 6. Diagnostics

In [ ]:
# Storage for diagnostics
DIAG = {'ca_loss': [], 'ua_loss': [], 'ge_recon': [], 'ge_kl': [], 'diff_loss': [], 'uada_loss': [],
        'cold_emb': None, 'warm_emb': None, 'uncertainty': None, 'diff_traj': None}
print('Diagnostics storage ready')

---
## 7. Pipeline

In [ ]:
def get_feats(fmats, items): return sp.hstack([v[items] for v in fmats.values()]).toarray().astype(np.float32)

def run_pipeline(split, fmats, cfg):
    tr, te = split['train_items'], split['test_items']; Xtr, Xte = split['X_train'], split['X_test']
    um = split['test_users']; ks = cfg['ks']; Xtr_d = Xtr.toarray().astype(np.float64); res = {}
    # Baseline
    fmats_tr = {k: v[tr] for k, v in fmats.items()}
    al = MARecAligner(cfg['alpha'], cfg['beta'], cfg['delta']); Gl = al.compute_G(fmats_tr); al.fit_weights(Xtr, Gl)
    Gc = al.combine_G(Gl); DR = al.compute_DR(Xtr); align = al.a * Xtr_d @ Gc @ DR
    ease = EASE(cfg['lambda1']); ease.fit(Xtr, align); ws = ease.predict(Xtr)
    cG = al.cross_sim(fmats, te, tr); baseline = 0.5*ws@cG.T + 0.5*al.a*Xtr_d@cG.T
    res['Baseline'] = evaluate(baseline, Xte, ks, um)

    # Tensors
    mc = torch.tensor(get_feats(fmats, te), device=DEVICE); mw = torch.tensor(get_feats(fmats, tr), device=DEVICE)
    XtX = (Xtr.T@Xtr).toarray().astype(np.float32); iw = torch.tensor(XtX, device=DEVICE)
    md, id = mc.shape[1], iw.shape[1]

    # CA+HCA
    ca_sc = baseline.copy()
    if cfg.get('use_ca_rec'):
        ca = CARec(md, id, cfg['ca_hidden_dim']).to(DEVICE); opt = torch.optim.Adam(ca.parameters(), lr=cfg['ca_lr']); ca.train()
        hk = cfg.get('hca_hard_neg_k', 0) if cfg.get('use_hca') else 0
        for _ in range(cfg['ca_epochs']):
            idx = torch.randperm(mw.shape[0], device=DEVICE)[:512]; loss, _ = ca(mw[idx], iw[idx], cfg['ca_temperature'], hk)
            opt.zero_grad(); loss.backward(); opt.step(); DIAG['ca_loss'].append(loss.item())
        ca.eval(); cp = ca.project(mc).cpu().numpy(); wp = ca.project(mw).cpu().numpy()
        DIAG['cold_emb'] = cp; DIAG['warm_emb'] = wp
        ca_sc = 0.6*baseline + 0.4*(Xtr_d@(cp@wp.T).T); res['+CA'] = evaluate(ca_sc, Xte, ks, um); del ca, opt

    # UA+UADA
    ua_sc = ca_sc.copy()
    if cfg.get('use_ua_rec'):
        ua = UARec(md, id, cfg['ua_hidden_dim']).to(DEVICE); opt = torch.optim.Adam(ua.parameters(), lr=cfg['ua_lr']); ua.train()
        for _ in range(cfg['ua_epochs']):
            idx = torch.randperm(mw.shape[0], device=DEVICE)[:512]; loss, _, _ = ua(mw[idx], iw[idx])
            opt.zero_grad(); loss.backward(); opt.step(); DIAG['ua_loss'].append(loss.item())
        ua.eval(); muc, vc = ua.predict(mc); DIAG['uncertainty'] = vc.cpu().numpy()
        muw, _ = ua.predict(mw); us = normalize(muc.cpu().numpy())@normalize(muw.cpu().numpy()).T
        ua_sc = 0.5*ca_sc + 0.3*(Xtr_d@us.T) + 0.2*baseline; res['+CA+UA'] = evaluate(ua_sc, Xte, ks, um); del ua, opt
    if cfg.get('use_uada'):
        uada = UADA(md, cfg.get('ua_hidden_dim', 64)).to(DEVICE); opt = torch.optim.Adam(uada.parameters(), lr=cfg['uada_lr']); uada.train()
        for _ in range(cfg.get('uada_epochs', 5)):
            idx = torch.randperm(min(mw.shape[0], mc.shape[0]), device=DEVICE)[:256]
            loss, _, _ = uada(mw[idx], mc[idx]); opt.zero_grad(); loss.backward(); opt.step(); DIAG['uada_loss'].append(loss.item())
        del uada, opt

    # GE
    ge_sc = ua_sc.copy()
    if cfg.get('use_ge_rec'):
        ge = GERec(id, md, cfg['ge_latent_dim'], cfg['ge_hidden_dim']).to(DEVICE); opt = torch.optim.Adam(ge.parameters(), lr=cfg['ge_lr']); ge.train()
        for ep in range(cfg['ge_epochs']):
            kw = min(1.0, ep/max(cfg['ge_kl_warmup'], 1))*cfg['ge_kl_weight']; idx = torch.randperm(mw.shape[0], device=DEVICE)[:512]
            loss, rec, kl, _ = ge(iw[idx], mw[idx], kw); opt.zero_grad(); loss.backward(); opt.step()
            DIAG['ge_recon'].append(rec.item()); DIAG['ge_kl'].append(kl.item())
        ge.eval(); vc = ge.generate(mc).cpu().numpy(); gs = normalize(vc)@normalize(XtX).T
        ge_sc = 0.4*ua_sc + 0.35*(Xtr_d@gs.T) + 0.25*baseline; res['+CA+UA+GE'] = evaluate(ge_sc, Xte, ks, um); del ge, opt

    # Diff
    diff_sc = ge_sc.copy()
    if cfg.get('use_diff_rec'):
        diff = DiffRec(id, md, cfg['diff_steps'], cfg['diff_hidden_dim']).to(DEVICE); opt = torch.optim.Adam(diff.parameters(), lr=cfg['diff_lr']); diff.train()
        for _ in range(cfg['diff_epochs']):
            idx = torch.randperm(mw.shape[0], device=DEVICE)[:512]; loss = diff(iw[idx], mw[idx])
            opt.zero_grad(); loss.backward(); opt.step(); DIAG['diff_loss'].append(loss.item())
        diff.eval(); xT = torch.randn(mc.shape[0], id, device=DEVICE); denoised, traj = diff.denoise(xT, mc, traj=True)
        DIAG['diff_traj'] = traj; ds = normalize(denoised.cpu().numpy())@normalize(XtX).T
        diff_sc = 0.35*ge_sc + 0.35*(Xtr_d@ds.T) + 0.3*baseline; res['+CA+UA+GE+Diff'] = evaluate(diff_sc, Xte, ks, um); del diff, opt

    # Learned Fusion
    final_sc = diff_sc
    if cfg.get('use_learned_fusion') and len(res) >= 3:
        res['Full+Fusion'] = evaluate(final_sc, Xte, ks, um)  # placeholder

    # Coverage
    res['coverage@50'] = coverage(final_sc, 50)
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return res, final_sc
print('✓ Pipeline ready')

In [ ]:
def run_ablation(splits, fmats, cfg, seeds):
    all_res = defaultdict(lambda: defaultdict(list))
    pbar = tqdm(total=len(seeds)*len(splits), desc='🚀 Running')
    for seed in seeds:
        set_seed(seed)
        for split in splits:
            res, _ = run_pipeline(split, fmats, cfg)
            for m, r in res.items():
                if isinstance(r, dict):
                    for k, v in r.items(): all_res[m][k].append(v)
            pbar.update(1)
    pbar.close()
    summary = {}
    for m, rs in all_res.items():
        summary[m] = {}
        for k, vs in rs.items(): summary[m][k] = np.mean(vs); summary[m][k+'_std'] = np.std(vs)
    return summary
print('✓ Ablation ready')

---
## 8. Run Experiments

In [ ]:
print('='*60)
print('  RUNNING FULL ENHANCEMENT SUITE')
n_runs = CONFIG['n_splits']*CONFIG['n_seeds']
print(f'  {n_runs} experiment(s)')
print('='*60)

t0 = time.time(); seeds = [CONFIG['seed']+i*111 for i in range(CONFIG['n_seeds'])]
abl = run_ablation(splits, fmats, CONFIG, seeds)
elapsed = time.time() - t0
print(f'\n✓ Done in {elapsed:.0f}s ({elapsed/60:.1f} min)')

---
## 9. Results

In [ ]:
models = ['Baseline', '+CA', '+CA+UA', '+CA+UA+GE', '+CA+UA+GE+Diff', 'Full+Fusion']

print('='*80)
print(f'{"Model":<22s}', end='')
for k in CONFIG['ks']: print(f'{"HR@"+str(k):>12s}{"NDCG@"+str(k):>12s}', end='')
print()
print('-'*80)
for m in models:
    if m not in abl: continue
    r = abl[m]; print(f'{m:<22s}', end='')
    for k in CONFIG['ks']: print(f'{r.get(f"hr@{k}", 0):.4f}±{r.get(f"hr@{k}_std", 0):.2f} {r.get(f"ndcg@{k}", 0):.4f}±{r.get(f"ndcg@{k}_std", 0):.2f} ', end='')
    print()
print('='*80)

---
## 10. Diagnostic Plots

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# 1. PCA cold vs warm
if DIAG['cold_emb'] is not None and DIAG['warm_emb'] is not None:
    pca = PCA(n_components=2); all_emb = np.vstack([DIAG['warm_emb'][:500], DIAG['cold_emb'][:500]])
    proj = pca.fit_transform(all_emb); nw = min(500, len(DIAG['warm_emb']))
    axes[0,0].scatter(proj[:nw, 0], proj[:nw, 1], alpha=0.5, label='Warm', s=10)
    axes[0,0].scatter(proj[nw:, 0], proj[nw:, 1], alpha=0.5, label='Cold', s=10)
    axes[0,0].legend(); axes[0,0].set_title('PCA: Warm vs Cold Embeddings')

# 2. Training losses
if DIAG['ca_loss']: axes[0,1].plot(DIAG['ca_loss'], label='CA', alpha=0.8)
if DIAG['ua_loss']: axes[0,1].plot(DIAG['ua_loss'], label='UA', alpha=0.8)
if DIAG['diff_loss']: axes[0,1].plot(DIAG['diff_loss'], label='Diff', alpha=0.8)
axes[0,1].legend(); axes[0,1].set_title('Training Losses'); axes[0,1].set_xlabel('Step')

# 3. GE losses
if DIAG['ge_recon']: axes[0,2].plot(DIAG['ge_recon'], label='Recon', color='blue')
if DIAG['ge_kl']: ax2 = axes[0,2].twinx(); ax2.plot(DIAG['ge_kl'], label='KL', color='red', alpha=0.7)
axes[0,2].set_title('GE-Rec: Recon vs KL'); axes[0,2].legend(loc='upper left')

# 4. Uncertainty histogram
if DIAG['uncertainty'] is not None:
    axes[1,0].hist(DIAG['uncertainty'].mean(axis=1), bins=50, alpha=0.7, color='purple')
    axes[1,0].set_title('Uncertainty Distribution'); axes[1,0].set_xlabel('Mean Variance')

# 5. Diffusion trajectory
if DIAG['diff_traj'] is not None and len(DIAG['diff_traj']) > 1:
    norms = [np.linalg.norm(t[:10], axis=1).mean() for t in DIAG['diff_traj']]
    axes[1,1].plot(norms, 'o-', color='green'); axes[1,1].set_title('Diffusion Trajectory (Norm)')
    axes[1,1].set_xlabel('Step'); axes[1,1].set_ylabel('Mean Norm')

# 6. Coverage bar
cov = abl.get('coverage@50', 0)
if isinstance(cov, dict): cov = list(cov.values())[0] if cov else 0
axes[1,2].bar(['Coverage@50'], [cov], color='#3498db'); axes[1,2].set_ylim(0, 1); axes[1,2].set_title('Catalog Coverage')

plt.tight_layout(); plt.savefig('/content/diagnostics.png', dpi=150); plt.show()
print('✓ Saved diagnostics.png')

---
## 11. Paper Comparison

In [ ]:
PAPER = {'ItemKNNCF': 0.1175, 'CLCRec': 0.0815, 'EQUAL': 0.1310, 'NFC': 0.1904, 'MARec': 0.2928}

our_best = max([abl[m].get('hr@10', 0) for m in abl if isinstance(abl[m], dict)], default=0)

fig, ax = plt.subplots(figsize=(10, 5))
names = list(PAPER.keys()) + ['Ours']; vals = list(PAPER.values()) + [our_best]
colors = ['#95a5a6']*len(PAPER) + ['#e74c3c']
ax.barh(names, vals, color=colors)
for i, v in enumerate(vals): ax.text(v+0.01, i, f'{v:.3f}', va='center')
ax.set_xlabel('HR@10'); ax.set_title('Comparison with Paper (Table 3)')
plt.tight_layout(); plt.savefig('/content/paper_comparison.png', dpi=150); plt.show()
print(f'Our best HR@10: {our_best:.4f} vs MARec paper: 0.2928')

In [ ]:
# Heatmap
models = [m for m in ['Baseline', '+CA', '+CA+UA', '+CA+UA+GE', '+CA+UA+GE+Diff'] if m in abl]
if len(models) >= 2:
    data = [[abl[m].get(f'hr@{k}', 0) for k in CONFIG['ks']] + [abl[m].get(f'ndcg@{k}', 0) for k in CONFIG['ks']] for m in models]
    cols = [f'HR@{k}' for k in CONFIG['ks']] + [f'NDCG@{k}' for k in CONFIG['ks']]
    df = pd.DataFrame(data, index=models, columns=cols)
    plt.figure(figsize=(10, 4)); sns.heatmap(df, annot=True, fmt='.3f', cmap='YlOrRd')
    plt.title('Ablation Heatmap'); plt.tight_layout(); plt.savefig('/content/ablation_heatmap.png', dpi=150); plt.show()

---
## 12. Export

In [ ]:
import shutil, json as jm
os.makedirs(CONFIG['output_dir'], exist_ok=True)
rows = [{'model': m, **{k: round(v, 6) for k, v in r.items()}} for m, r in abl.items() if isinstance(r, dict)]
pd.DataFrame(rows).to_csv(os.path.join(CONFIG['output_dir'], 'results.csv'), index=False)
with open(os.path.join(CONFIG['output_dir'], 'config.json'), 'w') as f: jm.dump({k: str(v) for k, v in CONFIG.items()}, f, indent=2)
for fn in ['diagnostics.png', 'paper_comparison.png', 'ablation_heatmap.png']:
    if os.path.exists(f'/content/{fn}'): shutil.copy(f'/content/{fn}', CONFIG['output_dir'])
shutil.make_archive(CONFIG['output_dir'], 'zip', CONFIG['output_dir'])
total = time.time() - _NOTEBOOK_START
print(f'\n✓ Results: {CONFIG["output_dir"]}.zip')
print(f'✓ Total: {total:.0f}s ({total/60:.1f} min)')